In [81]:
%pip install --quiet pydantic chromadb langchain langchain_experimental langchain_openai langchain_community

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [82]:
from pathlib import Path
import json

product_path = Path("../initial-training-sets/datasets/dataset_products.json")
product_images_path = Path("../initial-training-sets/datasets/images/")
product_data = json.loads(product_path.read_text())

from datetime import date
from uuid import UUID
from pydantic import BaseModel
from typing import List

class RawReview(BaseModel):
    id: UUID
    review_ref: str
    product_id: UUID
    review_content: str
    review_title: str
    date_written: date
    product_asin: str
    helpful_count: int
    rating_given: int
    review_page_url: str

class RAWProduct(BaseModel):
    id: UUID
    name: str
    description: str
    product_asin: str
    overall_ratings: float
    total_customers_that_rated: int
    price: float
    currency: str
    category: str
    sub_category: str
    product_page_url: str
    image_url: str
    reviews: List[RawReview]
    

from langchain_core.documents import Document

class ProductCombinedInformation:
    
    product: RAWProduct
    
    def __init__(self, product: RAWProduct) -> None:
        self.product = product
        
    def get_document(self):
        return Document(page_content=self.info(), id=str(self.product.id), metadata={
            "product_id": self.product.id,
            "product_asin": self.product.product_asin,
            "image_url": self.product.image_url,
            "name": self.product.name
        })
    
    
    def info(self):
        return (
            f"Product ID: {str(self.product.id).strip()} \n"
            f"Product Name: {str(self.product.name).strip()} \n"
            f"Product Description: {str(self.product.description).strip()} \n"
            f"Product Asin: {self.product.product_asin} \n"
            f"Overall Ratings {self.product.overall_ratings} \n"
            f"Total Customers that rated: {self.product.total_customers_that_rated} \n"
            f"Pric: {self.product.currency}{self.product.price} \n"   
        )
        
    def image_path(self):
        return f"{product_images_path}/{self.product.product_asin}.png"
    

In [293]:

product_summary_model_name = "llama3.1"
image_description_model_name = "llava"

# product_summary_model_name = "gpt-4o"
# image_description_model_name = "gpt-4o"
# text_embedding_model_name = ""

### PID: Product Image Description

In [85]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_community.chat_models import ChatOllama
from PIL import Image
from IPython.display import HTML, display
import base64

def encode_image(image_path: str):
    """Returns the base64 string for the image"""
    
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def describe_image(img_base64, prompt, model_name: str, image_format: str = "png"):
        
    
    """
    Describes the image provided. The prompt is used to guide the model on what
    to do with the description and what it should do.

    Raises:
        ValueError: If the wrong model name is provided.
        Exception: If any error occur

    Returns:
        str: The description generated for the image
    """
    
    if model_name not in [ "llava", "gpt-4o" ]:
        raise ValueError("Please provide either llava or gpt-4o")
    
    image_url = f"data:image/{image_format};base64,{img_base64}"
    
    if model_name == "gpt-4o":
        chat = ChatOpenAI(model=model_name, max_tokens=1024, temperature=0)
        msg = chat.invoke([
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_url
                        }
                    }
                ]
            )
        ])
    elif model_name == "llava":
        chat = ChatOllama(model=model_name, num_ctx=1024)
        msg = chat.invoke([
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_url
                        }
                    }
                ]
            )
        ])
    else:
        raise Exception("Unknown model name.")
        
    return msg.content


def plt_img_base64(img_base64):
  """Display base64 encoded string as image"""

  # Create an HTML img tag with the base64 string as the source
  image_html = f'<img width="200px" height="200px" src="data:image/jpeg;base64,{img_base64}" />'
  display(HTML(image_html))

### PCI: Product Combined Info

In [86]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def summarise_product_info(product_info: str, model_name: str):
    
    if model_name not in [ "llava", "gpt-4o", "llama3", "llama3.1", "phi3", "mistral" ]:
        raise ValueError("Please provide either llava or gpt-4o")
    
    prompt_template = ("You are an assistant tasked with summarizing tables and "
                    "text for retrieval. These summaries will be embedded and used " 
                    "to retrieve the raw text. Give a concise summary of the product. "
                    "Ensure you include important details "
                    "that is well optimized for retrieval.\n"
                    f"Product: {product_info}\n"
                    "Your response should start with the product name."
                    )
    
    prompt = ChatPromptTemplate.from_template(prompt_template)
    
    if model_name == "gpt-4o":
        model = ChatOpenAI(model=model_name, max_tokens=1024, temperature=0)
    else:
        model = ChatOllama(model=model_name, temperature=0)
    
        
    # { "product_info": RunnablePassthrough() } is same as lambda x: x
    # basically do not modify what the user entered, pass into the key product_info
    summarise_chain = { "product_info": RunnablePassthrough() } | prompt | model | StrOutputParser()
    summary = summarise_chain.invoke(product_info)
    return summary
    

In [88]:
max_products_per_category = 20

product_data.keys()

dict_keys(['MEN_FASHION_WITH_REVIEWS', 'SKIN_CARE_WITH_REVIEWS', 'WOMEN_FASHION_NO_REVIEWS'])

### Men Fashion

In [89]:
products = product_data['MEN_FASHION_WITH_REVIEWS']['products'][0:max_products_per_category]
combined_product_infos = [ ProductCombinedInformation(product=RAWProduct.model_validate(product)) for product in products ]

# Summarise Product Info = PCIS
product_info_summaries = [summarise_product_info(product_info.info(), model_name=product_summary_model_name) for product_info in combined_product_infos]


# for pis in product_info_summaries:
#     print(pis, "\n\n")

In [90]:

# Describe the Product Image = PID
image_prompt = f"""
Describe and summarise the characteristics of the product you are looking at. Start your response with: `The image is a product ... `. Return back plain text no markdown.
"""
product_image_descriptions = [describe_image(encode_image(image_path=product_info.image_path()), prompt=image_prompt, model_name=image_description_model_name) for product_info in combined_product_infos ]

for index, pid in enumerate(product_image_descriptions):
    print(index, pid, "\n\n")

0  The image is a product display showcasing three different color variants of what appears to be a sports shoe.  


1  The image is a product photo of a long-sleeve flannel shirt with a button-up front. It has two pockets on the chest, suggesting a practical design for carrying small items or for use in work environments. The shirt appears to have a casual yet functional style, suitable for both work and leisure activities.  


2  The image is a product display showcasing several pairs of men's underwear. There are three pairs visible in the image, all with the same branding: "CALVIN KLEIN". Each pair features a waistband with the text "CALVIN KLEIN" and the style appears to be a boxer brief. The colors of the underwear vary; one is black, another is dark grey, and the third is navy blue. The design suggests a classic and recognized brand in casual wear.  


3  The image is a product advertisement showcasing a set of socks. These socks feature a design that resembles the appearance of

In [91]:
product_info_summaries[2], product_image_descriptions[2]

('Here is a concise summary of the product, optimized for retrieval:\n\n**Men Boxer Short Trunks Stretch Cotton Pack of 3**\n\n* **Product Details**: Calvin Klein boxer shorts with CK branding, designed for everyday wear and sports.\n* **Material Composition**: 95% Cotton, 5% Elastane (Spandex).\n* **Care Instructions**: Machine Washable.\n* **Key Features**:\n\t+ Comfortable fit due to stretchy fabric.\n\t+ Classic elastic waistband with logo.\n\t+ Soft and breathable cotton stretch material.\n* **Brand Information**: Calvin Klein lifestyle brand, known for minimalist aesthetics and timeless designs.\n* **Product Specifications**:\n\t+ Package Dimensions: 19.56 x 13.21 x 3.3 cm; 100 Grams.\n\t+ Date First Available: June 18, 2023.\n\t+ Manufacturer: Calvin Klein.\n\t+ Item Model Number: 0000U2662G.\n* **Customer Reviews**: 4.6 out of 5 stars (36,088 ratings).\n* **Price**: £28.0.\n\nThis summary includes the most important details about the product, making it easy to retrieve and comp

### Create the Multi Vector Retriever

In [341]:
# Vector Store for Document Embeddings
from langchain_chroma import Chroma
from langchain_experimental.open_clip import OpenCLIPEmbeddings
from langchain_openai import OpenAIEmbeddings

from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

from chromadb.errors import InvalidDimensionException

# OpenCLIP: https://github.com/mlfoundations/open_clip
# Larger and more performant
# model_name = "ViT-g-14"
# checkpoint = "laion2b_s34b_b88k"

# # Smaller less performant
# model_name = "ViT-B-32"
# checkpoint = "laion2b_s34b_b79k"

# # Default
model_name = "ViT-H-14"
checkpoint = "laion2b_s32b_b79k"

text_embedding_model_name = "nomic-embed-text"

# for nomic-embed, embed_instruction is `search_document` to embed documents for RAG and `search_query` to embed the question
underlying_embedding = OllamaEmbeddings(model=text_embedding_model_name, embed_instruction="search_document", query_instruction="search_query")
underlying_embedding = OpenAIEmbeddings()
reset_collection = True

store = LocalFileStore("./embeddings-cache")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=underlying_embedding, document_embedding_cache=store, namespace=underlying_embedding.model
)


collection_name = f"fashion_store_mrag_v_{underlying_embedding.model}"
try:
    products_vectorstore = Chroma(
        collection_name=collection_name ,
        embedding_function=cached_embedder,
        # https://docs.trychroma.com/guides#changing-the-distance-function
        # Cosine, 1 means most similar, 0 means orthogonal, -1 means opposite
        collection_metadata={"hnsw:space": "cosine"}, # l2 is the default
        # embedding_function=OpenCLIPEmbeddings(model=None, preprocess=None, tokenizer=None, model_name=model_name, checkpoint=checkpoint)
    )
except InvalidDimensionException:
    Chroma().delete_collection()
    products_vectorstore = Chroma(
        collection_name=collection_name ,
        embedding_function=cached_embedder,
        collection_metadata={"hnsw:space": "cosine"}, # l2 is the default
    )
    
if reset_collection:
    products_vectorstore.reset_collection()

In [313]:
import json
import os
from pathlib import Path

images_path = "../initial-training-sets/datasets/images"

from langchain.retrievers.multi_vector import MultiVectorRetriever, SearchType
from langchain.storage import InMemoryStore
from langchain_core.documents import Document

from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=4000)

n_docs_to_retrieve = 4
relevant_threshold_score = 0.8

def create_multi_vector_retriever(
        embeddings_vectorsotre, 
        product_combined_info_summaries: List[str],  # PCIS -> Product Combined Info Summary
        product_image_descriptions: List[str],  # PID
        product_combined_infos: List[ProductCombinedInformation] # PCI objects
    ):
    
    # Initialise the document storage layer
    document_store = InMemoryStore()
    id_key = "product_id"
    
    retriever = MultiVectorRetriever(
        vectorstore=embeddings_vectorsotre,
        docstore=document_store,
        id_key=id_key,
        search_type=SearchType.similarity_score_threshold,
        search_kwargs={
            # below works only for similarity_score_threshold search_type
            "score_threshold": relevant_threshold_score, # close to 0 means most dissimilar, 1 means most similar
            "k": n_docs_to_retrieve
        }
    )
    
    # Helper function to add documents into the vector and the doument store
    def add_documents(retriever: MultiVectorRetriever, summary_to_embeds: List[str], combined_product_infos: List[ProductCombinedInformation]):
        product_ids=[ str(product_info_obj.product.id) for product_info_obj in combined_product_infos ]
        data = list(zip(product_ids, summary_to_embeds, combined_product_infos))
               
        docs = []
        parent_docs_contents = []
        
        for single_item in data:
            product_id, content_to_embed, product_info = single_item
            for content_chunk in splitter.split_text(content_to_embed):
                docs.append(Document(
                    page_content=content_chunk,
                    metadata={
                        id_key: product_id,
                        "id": product_id,
                        "name": product_info.product.name,
                        "product_asin": product_info.product.product_asin
                    }
                ))
                parent_docs_contents.append(
                    Document(
                        page_content=product_info.info(),
                        metadata={
                            id_key: product_info.product.id,
                            "id": product_info.product.id,
                            "name": product_info.product.name,
                            "product_asin": product_info.product.product_asin
                        }
                    ) 
                )
        
        assert len(docs) == len(parent_docs_contents)
        
        retriever.vectorstore.add_documents(docs)
        retriever.docstore.mset(list(zip(product_ids, parent_docs_contents)))
        
        return retriever
    
    if product_combined_info_summaries:
        add_documents(
            retriever=retriever,
            summary_to_embeds=product_combined_info_summaries,
            combined_product_infos=product_combined_infos
        )
        
    if product_image_descriptions:
        add_documents(
            retriever=retriever,
            summary_to_embeds=product_image_descriptions,
            combined_product_infos=product_combined_infos
        )
        
    return retriever
        
        

In [342]:

retriever = create_multi_vector_retriever(
    embeddings_vectorsotre=products_vectorstore, 
    product_combined_info_summaries=product_info_summaries, 
    product_image_descriptions=product_image_descriptions,
    product_combined_infos=combined_product_infos, 
)

retriever_query = "I dark coloured casual pants" # description only in the image
retriever_query = "high tech glasses" # description only in the image
retriever_query = "queen" # description only in the image
retriever_query = "pants with zipper fly" # description only in the image

docs = retriever.invoke(retriever_query)

for doc in docs:
    print(doc.page_content)
    

print("----")
# retriever.vectorstore.similarity_search_with_relevance_scores(retriever_query, k=3)
docs = retriever.vectorstore.similarity_search_with_relevance_scores(retriever_query, k=n_docs_to_retrieve)
for doc, score in docs:
    print(score, score >= relevant_threshold_score , " - " , doc.metadata.get('product_asin'), doc.page_content)


Product ID: 696b7628-954d-4303-b773-c81cd0ad687e 
Product Name: for Men, Slim Fit, Stretch - Comfortable and Modern Mens Trousers with Stretch for Suits, Business, Casual, Golf, Office 
Product Description: Product details     Material composition     3% Elastane (stretch), 75% Viscose, 22% Polyester       Care instructions     Machine Wash       Closure type     Zipper       Rise style     Mid Rise      About this item   STYLISH COMFORTABLE AFFORDABLE: Our chinos trousers men are trousers with a stylish look and great functionality so you can enjoy excellent comfort when wearing them. These black pants men feature a moisture-wicking fabric   EXCEPTIONAL COMFORT: In contrast to ordinary mens chinos our trousers men are equipped with a comfortable stretch effect so the wearer experiences optimal comfort when wearing these mens stretch trousers   PANTS FOR EVERY SITUATION: With our stylish and comfortable mens linen trousers you always have high wearing comfort and optimal mobility wheth

### Synthesize Answer / Pass docs to LLM

In [343]:
from langchain_community.chat_models.ollama import ChatOllama


model = ChatOllama(model="llama3.1", temperature=0, num_ctx=1024)
model = ChatOpenAI(model="gpt-4o", temperature=0)


from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever

# Retrieval Chain: summarise history + current query/input to fetch relevant documents
history_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        SystemMessage(content="generate a search query to look up in order to get information relevant to the conversation."),
        ("user", "{input}"),
    ]
)
history_chain = create_history_aware_retriever(
  llm=model, retriever=retriever, prompt=history_prompt
)

# Answer Synthesising
system_message = """
                  You work as a stylish and Gen-Z customer assistance in a fashion store.
                  
                  Inventory Start
                  {context}
                  Inventory Ends
                  
                  Try to persuade the user to go ahead and click the buy button for one of these products. 
                  It will take them to an Amazon page where they can continue their purchase
                  
                  In situation where there are multiple products in the inventory, 
                  explain how each of them can be helpful
                  
                  There are times the inventory will be empty, assure the user 
                  we will get the product that would match their query if it is something a 
                  fashion store can provide as a service.
                """
prompt = ChatPromptTemplate.from_messages([
    ("system", system_message ),
    MessagesPlaceholder(variable_name="chat_history"), # empty for now
    ("human", "{input}" ),
])

# a chain that allows us pass a list of docs to a model
stuff_documents_and_prompts_chain = create_stuff_documents_chain(llm=model, prompt=prompt)

# a chain to use the retriever we have to retireve the relevant docs and then pass them to the chain that has the llm+prompt
chain = create_retrieval_chain(
  retriever=history_chain, # We can either pass a retriever or a chain
  combine_docs_chain=stuff_documents_and_prompts_chain
)

# user_question = "The summer is here, what item can I buy to add to my accessory? Glasses, hats etc"
user_question = "The summer is here, what item can I buy to add to my accessory?"

# The documents fetched does not meet our threshold for relevance scores, so it wil be empty. Inventory is empty!
docs = retriever.vectorstore.similarity_search_with_relevance_scores(user_question, k=n_docs_to_retrieve)
for doc, score in docs:
    print(score, score >= relevant_threshold_score , " - " , doc.metadata.get('product_asin'), doc.page_content)


chain.invoke({"input": user_question, "chat_history": []})

0.8050183653831482 True  -  B01B1D12B0 The image is a product display of a pair of sunglasses.
0.7729806303977966 False  -  B0BRNFVSCF The image is a product of an item of clothing, specifically a grey sweatshirt or hoodie. It features the word "ESSENTIALS" printed on the front in white capital letters. The hoodie appears to be casual wear and has long sleeves, a kangaroo pocket on the left chest area, and a hood for added functionality.
0.7648344039916992 False  -  B09HXLDG5B The image is a product advertisement featuring a person wearing dark-colored casual pants, which appear to be high-waisted with a slim fit. The pants have a zipper fly and two front pockets. They are displayed in a way that shows the side seam and waistband design. The person is also wearing a white shirt and black shoes, suggesting a smart-casual style. The background of the image is plain, emphasizing the product.
0.7647953629493713 False  -  B09HXLDG5B The image is a product advertisement featuring a pair of b

{'input': 'The summer is here, what item can I buy to add to my accessory?',
 'chat_history': [],
 'context': [Document(metadata={'product_id': UUID('96f87d3b-5202-4ada-8b60-a1326719302f'), 'id': UUID('96f87d3b-5202-4ada-8b60-a1326719302f'), 'name': "Men's Retro Driving Polarized Sunglasses Man Al-Mg Metal Frame Ultra Light UV400 CAT 3 CE", 'product_asin': 'B01B1D12B0'}, page_content="Product ID: 96f87d3b-5202-4ada-8b60-a1326719302f \nProduct Name: Men's Retro Driving Polarized Sunglasses Man Al-Mg Metal Frame Ultra Light UV400 CAT 3 CE \nProduct Description: Product details     Care instructions     Far away from seawater or corrosive drugs,otherwise lens coating got damage.       Country of origin     Hong Kong      About this item   WHY CHOOSE US? -【Say goodbye to poor Polarized Lens】Low-quality lenses can cause lasting harm to your eyes. Our mens sun glasses are committed to using premium materials. Presently, ATTCL PrismX HD Polarized Lens incorporates an innovative PrismX layer, 

#### Thoughts on improving our Retriever

In a situation with the question below, we will need to introduce a multi-query to ask the questions in different ways we can have more products to 

Question: "The summer is here, what item can I buy to add to my accessory?"

Adding hats and glasses will improve the answer as shown below

In [344]:

user_question = "The summer is here, what item can I buy to add to my accessory? Glasses, hats etc"

# The documents fetched does not meet our threshold for relevance scores, so it wil be empty. Inventory is empty!
docs = retriever.vectorstore.similarity_search_with_relevance_scores(user_question, k=n_docs_to_retrieve)
for doc, score in docs:
    print(score, score >= relevant_threshold_score , " - " , doc.metadata.get('product_asin'), doc.page_content)


chain.invoke({"input": user_question, "chat_history": []})

0.8269434571266174 True  -  B01B1D12B0 The image is a product display of a pair of sunglasses.
0.7963172197341919 False  -  B01B1D12B0 The image is a product advertisement featuring a pair of glasses. The glasses have a distinctive design, with the arms appearing to be folded inwards rather than outwards, creating an unusual and possibly artistic appearance. This unique design feature suggests that the glasses may have been engineered to minimize their footprint when not in use. They also seem to incorporate some level of smart technology or a high-tech aesthetic, as indicated by the visible electronic components on the arms. The frame of the glasses has a sleek and modern look, with what appears to be a glossy finish. Overall, the product is presented as a fashionable accessory that combines style with potential functionality.
0.7892106771469116 False  -  B01B1D12B0 **Men's Retro Driving Polarized Sunglasses**

This product is a pair of men's retro driving polarized sunglasses made fr

{'input': 'The summer is here, what item can I buy to add to my accessory? Glasses, hats etc',
 'chat_history': [],
 'context': [Document(metadata={'product_id': UUID('96f87d3b-5202-4ada-8b60-a1326719302f'), 'id': UUID('96f87d3b-5202-4ada-8b60-a1326719302f'), 'name': "Men's Retro Driving Polarized Sunglasses Man Al-Mg Metal Frame Ultra Light UV400 CAT 3 CE", 'product_asin': 'B01B1D12B0'}, page_content="Product ID: 96f87d3b-5202-4ada-8b60-a1326719302f \nProduct Name: Men's Retro Driving Polarized Sunglasses Man Al-Mg Metal Frame Ultra Light UV400 CAT 3 CE \nProduct Description: Product details     Care instructions     Far away from seawater or corrosive drugs,otherwise lens coating got damage.       Country of origin     Hong Kong      About this item   WHY CHOOSE US? -【Say goodbye to poor Polarized Lens】Low-quality lenses can cause lasting harm to your eyes. Our mens sun glasses are committed to using premium materials. Presently, ATTCL PrismX HD Polarized Lens incorporates an innovat

#### What if we have multiple products?

In [345]:

user_question = "I need to buy new trousers! I need like three."

# The documents fetched does not meet our threshold for relevance scores, so it wil be empty. Inventory is empty!
docs = retriever.vectorstore.similarity_search_with_relevance_scores(user_question, k=n_docs_to_retrieve)
for doc, score in docs:
    print(score, score >= relevant_threshold_score , " - " , doc.metadata.get('product_asin'))


chain.invoke({"input": user_question, "chat_history": []})

0.8132283091545105 True  -  B09C5FHR99
0.8108167052268982 True  -  B09HXLDG5B
0.8018320798873901 True  -  B09HXLDG5B
0.7988615036010742 False  -  B09HXLDG5B


{'input': 'I need to buy new trousers! I need like three.',
 'chat_history': [],
 'context': [Document(metadata={'product_id': UUID('34e47362-144b-470c-8c96-8ccbf9e2ea8e'), 'id': UUID('34e47362-144b-470c-8c96-8ccbf9e2ea8e'), 'name': 'Mens Joggers Tracksuit Bottoms Men for Running Sports Lounge with Zip Pockets Elasticated Waist', 'product_asin': 'B09C5FHR99'}, page_content="Product ID: 34e47362-144b-470c-8c96-8ccbf9e2ea8e \nProduct Name: Mens Joggers Tracksuit Bottoms Men for Running Sports Lounge with Zip Pockets Elasticated Waist \nProduct Description: Product details     Material composition     70% Cotton, 25% Polyester, 5% Elastane       Care instructions     Hand Wash or Machine Wash       Closure type     Drawstring       Pocket style     Zipper Pocket      About this item   Safe Zip Pockets: Mens Joggers have 2 handy zip pockets each side, which provide safe space for storing for mobile phones,keys,wallets,etc.   Mens Joggers Bottoms: are ideal for home, walk, camping, jogging,